### IMPORTING REQUIRED LIBRARIES

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import datetime
from datetime import date
import numpy as np
import pandas as pd
import re
import os
import json
#import tiktoken # for token counting
import numpy as np
from collections import defaultdict
import openai
from openai import OpenAI
import ast

from llm_files.llm_response import *
from llm_files.llm_settings import *
from investment_analysis import *
from parameters import *


### STATIC INPUTS

In [18]:
tools_list=['calculate_roe', 'calculate_cashflow_benefit']

### TESTING THE FIRST LLM ANSWER

In [18]:
user_prompt = 'Assume that I buy a house in 1 year, sell it in 10 years and buy straight after that'
example = [{'action' : 'buy', 'year': 1}, {'action' : 'sell', 'year': 10}, {'action' : 'buy', 'year': 11} ]
adjusted_prompt = f"""Please convert the user request into a structured format of transactions, represented as a list of dictionaries. The request is: {user_prompt}
Each transaction is a dictionary with two keys: 'action' and 'year'.
'action' should be a single word describing the transaction (e.g., 'buy', 'sell'). Use an underscore to combine words if necessary.
'year' refers to the specific year the action takes place, not the number of years after the previous action.
If buying and selling happen in consecutive years, ensure there is at least a one-year gap between selling and buying again.
Please only returns the list of dictionaries as output

For example if I buy a house in 1 year, sell it in 10 years and buy again 1 year after that the output should be {example}

    """

response = client.chat.completions.create(
        model='gpt-4-1106-preview',
        messages=[{"role": "user", "content": user_prompt}],
        temperature=0,
        max_tokens=150,  # Adjust as necessary
        top_p=0,
        frequency_penalty=0,
        presence_penalty=0
    )
response.choices[0].message.content


"When you're considering buying a house with the intention to sell it in the future and then purchase another one, there are several factors to consider. Here's a general outline of what the process might involve, assuming you buy a house in one year, sell it in ten years, and then buy another one immediately after:\n\n**Year 1: Buying the First House**\n\n1. **Budgeting and Financing**: Determine your budget, including how much you can afford for a down payment and what mortgage terms you can secure. Get pre-approved for a mortgage to make your offer more attractive to sellers.\n\n2. **House Hunting**: Work with a real estate agent to find a house that meets your needs and is within your budget. Consider factors like location,"

### TESTING THE LLM ANSWER CHAIN

In [14]:
user_prompt = 'Assume the same real estate transactions as before'
updated_parameters_dict = initial_parameters
model_finetuned = 'ft:gpt-3.5-turbo-1106:de-martinos::8pGlpE1P'
model_general='gpt-4-turbo-preview'

generate_response(user_prompt = user_prompt, model_finetuned = model_finetuned, 
                  model_general = model_general, updated_parameters_dict = updated_parameters_dict)

True
['{', '\t"transactions": [', '\t\t{"action": "buy", "year": 2020},', '\t\t{"action": "sell", "year": 2021},', '\t\t{"action": "rent_out", "year": 2022}', '\t],', '\t"parameters": {', '\t\t', '\t},', '\t"functions": {', '\t\t', '\t}', '}']


IndexError: list index out of range

### TEST ELEMENTS EXTRACTION

In [7]:
user_prompt = 'Calculate the cash flow benefit if I buy a house in 3 years, sell it after 7 years, buy again straight after and sell in 45 years. I want to evaluate the investment after 10 years. Assume a house value of 400000 EUR.'
transactions_test, updated_parameters_test, function_dict_test = extract_transactions_and_parameters(user_prompt, model = 'ft:gpt-3.5-turbo-1106:de-martinos::8pGlpE1P')

In [59]:
function_dict_test['function']

'calculate_roe'

### TEST THE CODE TO ASSESS IF THE MODEL SHOULD BE USED

In [50]:
def assess_if_tool_is_needed(user_prompt, parameters_list, tools_list):

    adjusted_user_promt = f"""Given the user's request: '{user_prompt}', does it imply a real estate transaction or discuss relevant parameters or tools related to real estate investment analysis? Answer with 'True' or 'False' only."""
    
    # adjusted_user_promt = f"""if in the {user_prompt} there is any reference to this list of parameters: {parameters_list} or to this tools : {tools_list}
    #                             or any reference to real estate transactions such as buy, sell or rent a house then answer True otherwise False. You 
    #                             can only return the string True or the string False, nothing else"""
    
    llm_answer = query_llm(adjusted_user_promt, model = 'gpt-4-1106-preview')
    llm_answer_bolean = ast.literal_eval(llm_answer)
    
    return llm_answer_bolean

In [11]:
user_prompt='ciao'
assess_if_tool_is_needed(user_prompt=user_prompt)

False

### TESTING THE COMPLEX LLM REPSONSE

In [17]:
# Initialize state for transactions and parameters
state = {
    "transactions": [],
    "parameters": initial_parameters,
    "function_dict" : {'function_name':'calculate_roe', 'output_text' : 'the return on equity is  '}
}

function_dispatch = {
                            'calculate_roe': calculate_roe,
                            'calculate_cashflow_benefit' : calculate_cashflow_benefit 
                        # Add other functions here as needed
                                                            }

In [21]:
def handle_user_input(user_prompt, model_finetuned, model_general, parameters_list, tools_list, function_dispatch):
    global state  # Ensure state is accessible and updatable
    
    # Add user input to conversation history
    
    #conversation.add_user_input(user_prompt)  # Context + user input

    # Generate output based on the current input and conversation context
    tool_to_be_used = assess_if_tool_is_needed(user_prompt=user_prompt)

    print(tool_to_be_used)
    if tool_to_be_used:
        # Process output to extract transactions and parameters
        transactions, updated_parameters, function_dict = extract_transactions_and_parameters(user_prompt, model = model_finetuned)        
        
        # Check if transactions are empty and use the last state if so
        if not transactions:
            transactions = [state['transactions'][-1]] if state['transactions'] else []

        new_initial_parameters = create_updated_parameters_dict(updated_parameters = updated_parameters, initial_parameters_dict = state["parameters"])

        # Update state with new transactions and parameters, if provided
        if transactions:
            state["transactions"] = transactions
        if updated_parameters:
            state["parameters"] = new_initial_parameters  # assuming you want to replace parameters
        if function_dict:
            state['function_dict'] = function_dict

        # Add system output to conversation history
        #conversation.add_system_response(output)
        

            
        function_to_be_used = function_dispatch[state['function_dict']['function']]
        
        answer = f"{state['function_dict']['output_text']}{function_to_be_used(state['transactions'], state['parameters'])}"


        #Example Functions to be used
        
        #conversation.add_system_response(f"ROE Calculation: {roe_result}")
        
        return answer
    
    else:
        standard_answer = query_llm(user_prompt, model = model_general)
        return standard_answer

In [23]:
user_prompt = 'What is the return on Equity if I buy a house in 1 year, sell in 10 year'
handle_user_input(user_prompt=user_prompt, model_finetuned='ft:gpt-3.5-turbo-1106:de-martinos::8pGlpE1P', model_general='gpt-4-turbo-preview'
                    , parameters_list=parameters_list, tools_list=tools_list, function_dispatch = function_dispatch)

True


KeyError: None

### TESTING THE STREAMLIT SETUP

In [78]:
user_prompt = 'Calculate the cashflow benefit if I buy a house in 1 year, sell it in 10 years, consider that the investment horizon is 10 years'
generate_response(user_prompt=user_prompt, model_finetuned='ft:gpt-3.5-turbo-1106:de-martinos::8pGlpE1P', 
                  model_general='gpt-4-turbo-preview', function_dispatch = function_dispatch)

True


'the cashflow benefit is 120062.0'

In [37]:
test_list = [17672, 17392, 17101, 16796, 16477, 16144, 15796, 15433, 15053, 14656, 14241, 13807, 13354, 12881, 12386, 11869, 11328, 10764, 10174, 9557, 8913, 8239, 7536, 6800, 6032, 5229, 4390, 3513, 2597, 1639, 639, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(test_list)

45

In [43]:
interest_payments_test = [17392, 17101, 16796, 16477, 16144, 15796, 15433, 15053, 14656, 14241, 13807, 13354, 12881, 12386, 11869, 11328, 10764, 10174, 9557, 8913, 8239, 7536, 6800, 6032, 5229, 4390, 3513, 2597, 1639, 639, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
print(len(interest_payments_test))
test_list == interest_payments_test


44


False